## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [38]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [39]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023854952B90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023854953B90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [40]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Suraj and I am a Chief AI Engineer")])

AIMessage(content="Hello Suraj, it's nice to meet you! As a Chief AI Engineer, I imagine you have a fascinating and challenging role.  \n\nWhat kind of AI projects are you currently working on?  \n\nI'm always eager to learn more about the cutting edge of AI development.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 23, 'total_tokens': 85, 'completion_time': 0.112727273, 'prompt_time': 0.00135526, 'queue_time': 0.267473269, 'total_time': 0.114082533}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--271cf4a3-5742-47d9-8238-b89d7bd3cc4d-0', usage_metadata={'input_tokens': 23, 'output_tokens': 62, 'total_tokens': 85})

In [41]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Suraj and I am a Chief AI Engineer"),
        AIMessage(content="Hello Suraj! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Suraj, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like me to remember about you? 😄  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 99, 'total_tokens': 140, 'completion_time': 0.074545455, 'prompt_time': 0.002828939, 'queue_time': 0.255129141, 'total_time': 0.077374394}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--28c7abdc-8c85-4f2f-b5fa-4b79d66e55f6-0', usage_metadata={'input_tokens': 99, 'output_tokens': 41, 'total_tokens': 140})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [42]:
!pip install langchain_community

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [44]:
config={"configurable":{"session_id":"chat1"}}

In [45]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Suraj and I am a Chief AI Engineer")],
    config=config
)

In [46]:
response.content

"Hello Suraj, it's a pleasure to meet you!\n\nThat's a fascinating role. As a Chief AI Engineer, I imagine you're involved in some cutting-edge work.  \n\nWhat kind of projects are you currently working on? I'm always eager to learn more about the exciting applications of AI.\n"

In [47]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Suraj.  \n\nYou told me at the beginning of our conversation!  😊  \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 107, 'total_tokens': 142, 'completion_time': 0.063636364, 'prompt_time': 0.00282606, 'queue_time': 0.25039967, 'total_time': 0.066462424}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6b1419eb-979e-404f-895e-c3e9e0b0d830-0', usage_metadata={'input_tokens': 107, 'output_tokens': 35, 'total_tokens': 142})

In [48]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [49]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you!  \n\nIs there anything I can help you with today?\n"

In [50]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, you told me a little while ago! 😊  \n\nDo you have any other questions for me, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [51]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [52]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Suraj")]})

AIMessage(content="Hi Suraj, it's nice to meet you! \n\nI'm ready to answer your questions to the best of my ability.  What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.00148039, 'queue_time': 0.25424307, 'total_time': 0.08148039}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--dcafafe4-ec3c-4773-8d2c-1916bf493759-0', usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76})

In [53]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [54]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Suraj")],
    config=config
)

response

AIMessage(content="Hi Suraj, it's nice to meet you!  I'm happy to help with any questions you have.  \n\nWhat can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 32, 'total_tokens': 72, 'completion_time': 0.072727273, 'prompt_time': 0.00147956, 'queue_time': 0.25014933, 'total_time': 0.074206833}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--625aecd1-6fb7-47d4-84c4-5586fce85454-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [55]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Suraj,  you told me! 😊  \n\n\nHow can I help you further?\n'

In [56]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [57]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Suraj")],"language":"Hindi"})
response.content

'नमस्ते सूरज! \n\nमुझे खुशी है कि आपने मुझसे जुड़ना चाहा। मैं आपकी मदद करने के लिए तैयार हूँ। \n\nआप क्या पूछना चाहते हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [58]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [59]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Suraj")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते सूरज! 😊 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप क्या जानना चाहते हैं? \n'

In [60]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [61]:
response.content

'आपका नाम सूरज है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [62]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\Suraj Khodade\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [63]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor? 😊\n"

In [64]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 equals. 😊  \n\n\n\n\n'

In [65]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [66]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or personal information about you. So I don't know your name.\n\nWould you like to tell me? 😊\n"

In [67]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\nWhat's the problem?  \n\n"